In [2]:
import pandas as pd

file = "Resources/purchase_data.csv"
pymoli = pd.read_csv(file)

pymoli.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Count the unique items in "SN" Column to determine the amount of players
pymolipc = pymoli["SN"].nunique()
totalpc = pd.DataFrame([{"Total Players": pymolipc}])
totalpc

,Total Players
0,576


In [4]:
#Purchasing analysis
#Find count of unique items on item name list
uniqueitems = pymoli["Item Name"].nunique()
#Find average price by taking the mean of price column 
avgprice = pymoli["Price"].mean(axis = 0)
#Number of purchases is the same as the amount of purchase ID's
numpurch = pymoli["Purchase ID"].count()
#Find total revenue by calculating the sum of the "Price" column
totalrev = pymoli["Price"].sum(axis= 0)
#Create a list of dictionaries
purchanaly_dict = [{"# of Unique Items": uniqueitems,
             "Average Price": avgprice,
              "# of Purchases": numpurch,
             "Total Revenue": totalrev}]
#Turn L of D's into a DF
purchanaly_df = pd.DataFrame(purchanaly_dict)
#Format "Average Price" "Total Revenue" columns into "$" values
purchanaly_df["Average Price"] = purchanaly_df["Average Price"].map('${:,.2f}'.format)
purchanaly_df["Total Revenue"] = purchanaly_df["Total Revenue"].map('${:,.2f}'.format)
#Display DF
purchanaly_df

,# of Unique Items,Average Price,# of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [26]:
#Gender Demographic
#Display "Gender" & "SN" columns as new DF
gendem = pymoli[["Gender", "SN"]]
#Drop any duplicates from this df
gendem1 = gendem.drop_duplicates(["Gender", "SN"])
#Group By gender and count the values in the SN field 
gendem2 = gendem1.groupby("Gender").count()
#Find percentage of players in each gender
gendempercent = gendem2["SN"]/gendem2["SN"].sum() *100
#Sort SN values from highest to lowest
gendem3 = gendem2.sort_values("SN", ascending=False)
#Add percentage of players in each gender as a column
gendem3["Percentage of Players"] = gendempercent
#Rename "SN" to "Total Count"
gendem4 = gendem3.rename(columns={"SN": "Total Count"})
#Format "Percentage of Players" into percentage
gendem4["Percentage of Players"] = gendem4["Percentage of Players"].map('{:.2f}%'.format)
#Display DF
gendem4

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [30]:
#Gender Purchase Data
#Make total count from Gender Demographics into variable for later
gencountacc = gendem4["Total Count"]
#Display "Gender", "Price", "SN  from pymoli as new DF
genpurch= pymoli[["Gender", "Price", "SN"]]
#Group DF by "Gender" and take the sum of the columns
genpurch1= genpurch.groupby(["Gender"]).sum()
#Create Three remaining columns
genpurch1["Total Count"] = pymoli["Gender"].value_counts()
genpurch1["Average Purchase Price"] = genpurch1["Price"]/genpurch1["Total Count"]
genpurch1["Avg Total Purchase Per Person"] = genpurch1["Price"]/gencountacc 
genpurch2 = genpurch1.rename(columns={"Price": 'Total Purchase Value'})
#reorder cloumns to match starterbook
genpurch3 = genpurch2[['Total Count',
 'Average Purchase Price','Total Purchase Value',
 'Avg Total Purchase Per Person']]
#Format 'Avg Purchase','Total Value' 'Average Total Purchase Per Person' columns into "$" values
genpurch3['Average Purchase Price'] = genpurch3['Average Purchase Price'].map('${:,.2f}'.format)
genpurch3['Total Purchase Value'] = genpurch3['Total Purchase Value'].map('${:,.2f}'.format)
genpurch3['Avg Total Purchase Per Person'] = genpurch3['Avg Total Purchase Per Person'].map('${:,.2f}'.format)
genpurch3.head()

,Total Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [31]:
#Age Demographics
#Find Max and Min of Age column
agemax = pymoli["Age"].max()
agemin = pymoli["Age"].min()
#Display "Age", "SN" from pymoli as new DF
agedem = pymoli[["Age", "SN"]]
#Drop Duplicate Values from field
agedem1 = agedem.drop_duplicates(["Age", "SN"])
#Group by "SN" to list the age of each gamer
agedem2 = agedem1.groupby(agedem1["SN"]).sum()
#Create bins to place age list into
bins = [0,9,14,19,24,29,34,39,45]
#Create labels for said bins
labels = ["<10","10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#use cut function to sort data into bins
age = pd.cut(agedem2["Age"],bins, labels=labels)
#turn bins into a DF (Age will change from actual age to age range they are in )
agedf = pd.DataFrame(age)
#reset index to default to drop "SN" values
agedf1 = agedf.reset_index(drop=True)
#Count the amount of times each value shows up in "Age" column
agedf2 = agedf1["Age"].value_counts()
#Convert agedf2 into DF
agedf3 = pd.DataFrame(agedf2)
agedf3 = agedf3.rename(columns={"Age": "Total Count"})
#Make the sum of "Age" into variable for later
count = agedf3["Total Count"].sum() 
#Calculate percentage by dividing each row in "Age" by count then and make it into a new column
agedf3["Percentage of Players"] = agedf3["Total Count"]/count * 100
#Re-Order the index from youngest to oldest
agedf4 =  agedf3.reindex(index = ["<10","10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])
#Format "Percentage of Players" into percentage
agedf4["Percentage of Players"] = agedf4["Percentage of Players"].map('{:.2f}%'.format)
#Display DF
agedf4


,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [32]:
#Purchasing Analysis Age
#Display "Age", "Price","SN" as DF
agepurch = pymoli[["Age", "Price","SN"]]
agepurch
#Make "Total Count" from agedf3 into variable for later
agecountacc = agedf3["Total Count"]
#Duplicate Bins and labels from Age Demographics
bins = [0,9,14,19,24,29,34,39,45]

labels = ["<10","10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
#Use cut function to bin the "Age" value from agepurch
purchage = pd.cut(agepurch["Age"],bins, labels=labels)
#turn bins into a DF (Age will change from actual age to age range they are in )
purchagedf = pd.DataFrame(purchage)
#Rename "Age" to "Age Group"
purchagedf1 = purchagedf.rename(columns={"Age":"Age Group"})
#combine agepurch and purchagedf1 as one single DF
merged = pd.concat([agepurch,purchagedf1], axis = 1, sort=False)
#Do two groupby's by "Age Group" (one count, and one sum) to create two new DF's
mergedar = merged.groupby("Age Group").count()
mergedar1 = merged.groupby("Age Group").sum()
#rename columns in margedar1 to avoid columns with same name in each DF
mergedar1 = mergedar1.rename(columns={"Age":"Age1", "Price": "Total Purchase Value"})
#combine mergedar and mergedar1 into one single DF
mergedfinal = pd.concat([mergedar,mergedar1], axis = 1, sort=False)
#Drop unnecessary columns from new DF
mergedfinal1 = mergedfinal.drop(columns={"Price", "SN", "Age1"})
#Rename "Age" to "Purchase Count"
mergedfinal1= mergedfinal1.rename(columns={"Age": "Purchase Count"})
#Create avgpurchprice as a variable by dividing mergedfinal1["Total Purchase Value"] by mergedfinal1["Purchase Count"]
avgpurchprice = mergedfinal1["Total Purchase Value"]/ mergedfinal1["Purchase Count"]
#Turn avgpurchprice into a new column named "Average Purchase Price"
mergedfinal1["Average Purchase Price"] = avgpurchprice
#Create avetotalper as a variable by dividing mergedfinal1["Total Purchase Value"] by agecountacc
avgtotalper = mergedfinal1["Total Purchase Value"] / agecountacc
#turn avgtotalper into a new column named "Avg Total Purchase per Person"
mergedfinal1["Avg Total Purchase per Person"] = avgtotalper
#Reorder columns to match starterbook
agepurchanalfinal = mergedfinal1[['Purchase Count','Average Purchase Price',
'Total Purchase Value','Avg Total Purchase per Person']]
#Format 'Average Purchase Price','Total Purchase Value','Avg Total Purchase per Person' into "$" values
agepurchanalfinal['Average Purchase Price'] = agepurchanalfinal['Average Purchase Price'].map('${:,.2f}'.format)
agepurchanalfinal['Total Purchase Value'] = agepurchanalfinal['Total Purchase Value'].map('${:,.2f}'.format)
agepurchanalfinal['Avg Total Purchase per Person'] = agepurchanalfinal['Avg Total Purchase per Person'].map('${:,.2f}'.format)
#Display DF
agepurchanalfinal


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [33]:
#TopSpenders
#Display "SN" & "Price" as DF
topspend = pymoli[["SN", "Price"]]
#Groupby topspend by "SN" and take the sum
topspend1 = topspend.groupby(["SN"]).sum()
#Sort DF by "Price" from largest to smallest
topspend2 = topspend1.sort_values(by = "Price", ascending =False)
#Turn price column into variable for later
price = topspend2["Price"]
#Count how many times each value appears in "SN" in pymoli DF and make into variable
purchcount = pymoli["SN"].value_counts()
#Calculate "Average Purchase Price" by dividing price  by purchcount and create new column
topspend2["Avg Purch Price"] = price / purchcount
#Create new column for purchcount named "Purchase Count"
topspend2["Purchase Count"] = purchcount
#Rename "Price" to "Total Value"
topspend3 = topspend2.rename(columns={"Price": "Total Value"})
#Reorder columns to match starterbook
topspend_final = topspend3[["Purchase Count", "Avg Purch Price", "Total Value"]]
#Format "Avg Purch Price" & "Total Value" to "$" values
topspend_final["Avg Purch Price"] = topspend_final["Avg Purch Price"].map('${:,.2f}'.format)
topspend_final["Total Value"] = topspend_final["Total Value"].map('${:,.2f}'.format)
topspend_final.head()

,Purchase Count,Avg Purch Price,Total Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [34]:
#Most Popular Items
#Display "Item ID","Item Name", "Price" as DF
popitem = pymoli[["Item ID","Item Name", "Price"]] 
#Groupby DF by "Item Name" and take the sum
popitem1 = popitem.groupby(["Item Name"]).sum() 
#Count how many time each value shows up in "Item Name" column in pymoli
# and save as a variable
count = pymoli["Item Name"].value_counts() 
#Turn count into new column and name "Purchase Count"
popitem1["Purchase Count"] = count
#Return "Item ID" column to correct value by dividing it by count
popitem1["Item ID"] = popitem1["Item ID"]/ count 
#Change "Item ID" from a float to an integer
intset = popitem1["Item ID"].astype(int)
#replace float with integer in "Item ID"
popitem1["Item ID"] = intset
#Duplicate "Item Name" and make it a regular column
popitem1["Item Name"] = popitem1.index
#Replace "Item Name" with "Item ID" as index
popitem2 = popitem1.set_index("Item ID")
#Create "Item Price" column by dividing popitem2["Price"] by popitem2["Purchase Count"]
popitem2["Item Price"] = popitem2["Price"]/popitem2["Purchase Count"]
#Sort DF by "Purchase Count" from Largest to Smallest
popitem3 = popitem2.sort_values(by="Purchase Count", ascending=False)
#Rename "Price" column to "Total Purchase Value"
popitem4 = popitem3.rename(columns={"Price": "Total Purchase Value"})
#Reorder columns to match starterbook
popitem5 = popitem4[["Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]
#Format "Item Price" and "Total Purchase Value" to "$" values
popitem5['Item Price'] = popitem5['Item Price'].map('${:,.2f}'.format)
popitem5["Total Purchase Value"] = popitem5["Total Purchase Value"].map('${:,.2f}'.format)
#Display DF
popitem5.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
132,Persuasion,9,$3.22,$28.99
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [35]:
#Most Profitable Items
#Sort DF by "Total Value" column from largest to smallest
profit = popitem4.sort_values(by=["Total Purchase Value"], ascending=False)
#Reorder the coolumns to match starterbook
profit1 = profit[["Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]
#Format "Item Price" and "Total Purchase Value" into "$" values
profit1['Item Price'] = profit1['Item Price'].map('${:,.2f}'.format)
profit1['Total Purchase Value'] = profit1['Total Purchase Value'].map('${:,.2f}'.format)
#Display DF
profit1.head()

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
